In [1]:
from ftplib import FTP_TLS

In [2]:
ftps = FTP_TLS('sf01a-aws-syd.ratecity.com.au')
ftps.login("salesforce", "tZyL!KoO6I2e")        
ftps.prot_p()      
ftps.retrlines('LIST')

drwxr-xr-x    2 502      502          4096 Feb 13  2017 logs
drwxr-xr-x    2 502      502         57344 Nov 15 20:10 output


'226 Directory send OK.'

In [3]:
ftps.cwd("/output")
gFile1 = open('2017-Oct-16_LastMonthOrders.csv', 'wb')
ftps.retrbinary('RETR' + ' 2017-Oct-16_LastMonthOrders.csv', gFile1.write)
    

'226 Transfer complete.'

In [4]:
import pandas as pd
df1=pd.read_csv('2017-Oct-16_LastMonthOrders.csv')

In [5]:
df1=df1.drop_duplicates().reset_index(drop=True)

In [6]:
df1.rename(columns={'Id': 'osli_id','Name' : 'osli_number' ,
                   'RC_Console_Product__r.RC_Product_ID__c':'product_uuid',
                   'RC_Console_Product__r.RC_Product_Type__c':'vertical',
                   'RC_Actual_Cicks__c':'actual_clicks','RC_No_of_CPA_actuals__c':'number_of_cpa',
                   'RC_No_of_CPAA_Actual__c':'number_of_cpaa',
                   'RC_No_of_CPL_Actual__c':'number_of_cpl','RC_CPA_Rate__c':'p_cpa',
                   'RC_CPAA_Rate__c':'p_cpaa','RC_CPL_Rate__c':'p_cpl','RC_Product_Url__c':'product_url',
                   'RC_Order_Line_Item__r.Order.OrderNumber':'order_number',
                   'RC_Order_Line_Item__r.Order.Account.Name':'company_name',
                   'RC_Order_Line_Item__r.OrderItemNumber':'order_product_number',
                   'RC_Order_Line_Item__r.Order.RC_SignedDate__c':'order_signed_date',
                   'RC_Order_Line_Item__r.PricebookEntry.Product2.Name':'order_product_name',
                   'RC_Order_Line_Item__r.RC_Start_Date__c':'date_start',
                   'RC_Order_Line_Item__r.RC_End_Date__c':'date_end',
                   'RC_Order_Line_Item__r.UnitPrice':'p_unit',
                   'RC_Order_Line_Item__r.Billed_Clicks__c':'billed_clicks',
                   'RC_Order_Line_Item__r.RC_Capping__c':'capped',
                   'RC_Order_Line_Item__r.RC_Budget_Consumption__c':'budget_consumption',
                   'RC_Order_Line_Item__r.RC_Budget__c':'budget',
                   'RC_Order_Line_Item__r.Order.RC_Agency__r.Name':'agency_name',
                   'RC_Order_Line_Item__r.Order.RC_Agency_Percentage__c':'agency_percentage',
                   'RC_Order_Line_Item__r.RC_Agency_Commission_Amount__c':'agency_commission',
                   'RC_Console_Product__r.Name':'console_product_name',
                   'RC_Order_Line_Item__r.Order.EffectiveDate':'order_effective_date',
                   'RC_Order_Line_Item__r.Order.EndDate':'order_end_date',
                   'RC_Order_Line_Item__r.Order.CompanyAuthorizedBy.Name':'authorized_by_name',
                   'RC_Order_Line_Item__r.Order.Owner.Name':'owner_name',
                   'RC_Order_Line_Item__r.Excess_Clicks__c':'excess_clicks',
                   'RC_Order_Line_Item__r.RC_Gross_Revenue__c':'gross_revenue'
                   
                  }, inplace=True)

In [7]:
columns = ['RC_Console_Product__c','RC_Order_Line_Item__r.OrderId','RC_Order_Line_Item__r.Order.IsReductionOrder']
df1.drop(columns, inplace=True, axis=1)

In [8]:
ftps.cwd("/output")
g3File = open('2017-Oct-16_LastMonthOrdersWithoutOSLs.csv', 'wb')
ftps.retrbinary('RETR' + ' 2017-Oct-16_LastMonthOrdersWithoutOSLs.csv', g3File.write)

'226 Transfer complete.'

In [9]:
wo_ogsl_1=pd.read_csv('2017-Oct-16_LastMonthOrdersWithoutOSLs.csv')

In [10]:
wo_ogsl_1.columns

Index(['OrderId', 'Order.OrderNumber', 'Order.Account.Name',
       'Order.EffectiveDate', 'Order.EndDate', 'OrderItemNumber',
       'Order.RC_SignedDate__c', 'PricebookEntry.Product2.Name',
       'Order.CompanyAuthorizedBy.Name', 'Order.Owner.Name',
       'RC_Start_Date__c', 'RC_End_Date__c', 'ListPrice', 'UnitPrice',
       'Billed_Clicks__c', 'Excess_Clicks__c', 'RC_Gross_Revenue__c',
       'RC_Capping__c', 'RC_Budget_Consumption__c', 'RC_Budget__c',
       'Order.RC_Agency__r.Name', 'Order.RC_Agency_Percentage__c',
       'RC_Agency_Commission_Amount__c', 'Order.IsReductionOrder'],
      dtype='object')

In [11]:
wo_ogsl_1.rename(columns={'Order.OrderNumber': 'order_number','Order.Account.Name':'company_name',
                        'Order.EffectiveDate':'order_effective_date','Order.EndDate':'order_end_date',
                        'OrderItemNumber':'order_product_number','Order.RC_SignedDate__c':'order_signed_date',
                        'PricebookEntry.Product2.Name':'order_product_name',
                        'Order.CompanyAuthorizedBy.Name':'authorized_by_name',
                        'Order.Owner.Name':'owner_name','RC_Start_Date__c':'date_start',
                        'RC_End_Date__c':'date_end','UnitPrice':'p_unit','Billed_Clicks__c':'billed_clicks',
                        'Excess_Clicks__c':'excess_clicks','RC_Gross_Revenue__c':'gross_revenue',
                        'RC_Capping__c':'capped','RC_Budget_Consumption__c':'budget_consumption',
                        'RC_Budget__c':'budget','Order.RC_Agency__r.Name':'agency_name',
                        'Order.RC_Agency_Percentage__c':'agency_percentage',
                        'RC_Agency_Commission_Amount__c':'agency_commission'
                       },inplace=True)

In [12]:
columns2=['OrderId','Order.IsReductionOrder']
wo_ogsl_1.drop(columns2, inplace=True, axis=1)

In [13]:
wo_ogsl_1["vertical"]=''

wo_ogsl_1["vertical"][wo_ogsl_1['order_product_name'].str.contains("Home Loans")] = "home loans"

wo_ogsl_1["vertical"][wo_ogsl_1['order_product_name'].str.contains("Car Loans")] = "car loans"

wo_ogsl_1["vertical"][wo_ogsl_1['order_product_name'].str.contains("Personal Loans")] = "personal loans"

wo_ogsl_1["vertical"][wo_ogsl_1['order_product_name'].str.contains("Term Deposits")] = "term deposits"

wo_ogsl_1["vertical"][wo_ogsl_1['order_product_name'].str.contains("Credit Cards")] = "credit cards"

wo_ogsl_1["vertical"][wo_ogsl_1['order_product_name'].str.contains("Car Insurance")] = "car insurance"

wo_ogsl_1["vertical"][wo_ogsl_1['order_product_name'].str.contains("Managed Funds")] = "managed funds"

wo_ogsl_1["vertical"][wo_ogsl_1['order_product_name'].str.contains("Savings A/C")] = "savings accounts"

wo_ogsl_1["vertical"][wo_ogsl_1['order_product_name'].str.contains("Transaction A/C")] = "transaction accounts"

wo_ogsl_1["vertical"][wo_ogsl_1['order_product_name'].str.contains("Transaction accounts")] = "transaction accounts"

wo_ogsl_1["vertical"][wo_ogsl_1['order_product_name'].str.contains("Pension Funds")] = "pension"

wo_ogsl_1["vertical"][wo_ogsl_1['order_product_name'].str.contains("Pension")] = "pension"

wo_ogsl_1["vertical"][wo_ogsl_1['order_product_name'].str.contains("Superannuation")] = "superannuation"

wo_ogsl_1["vertical"][wo_ogsl_1['order_product_name'].str.contains("Managed Funds")] = "managed funds"

wo_ogsl_1["vertical"][wo_ogsl_1['order_product_name'].str.contains("Online Share Trading")] = "online trading"

wo_ogsl_1["vertical"][wo_ogsl_1['order_product_name'].str.contains("Online")] = "online trading"

wo_ogsl_1["vertical"][wo_ogsl_1['order_product_name'].str.contains("Margin Loans")] = "margin loans"

wo_ogsl_1["vertical"][wo_ogsl_1['order_product_name'].str.contains("Foreign Exchange")] = "foreign exchange"

wo_ogsl_1["vertical"][wo_ogsl_1['order_product_name'].str.contains("Mortgage Insurance")] = "mortgage insurance"

wo_ogsl_1["vertical"][wo_ogsl_1['order_product_name'].str.contains("Home Insurance")] = "home insurance"

wo_ogsl_1["vertical"][wo_ogsl_1['order_product_name'].str.contains("Bad Credit")] = "bad credit"

wo_ogsl_1["vertical"][wo_ogsl_1['order_product_name'].str.contains("Business Loans")] = "business loans"

wo_ogsl_1["vertical"][wo_ogsl_1['order_product_name'].str.contains("Health Product")] = "health product"

wo_ogsl_1["vertical"][wo_ogsl_1['order_product_name'].str.contains("Income Protection Insurance")] = "income protection insurance"

wo_ogsl_1["vertical"][wo_ogsl_1['order_product_name'].str.contains("Life Insurance")] = "life insurance"

wo_ogsl_1["vertical"][wo_ogsl_1['order_product_name'].str.contains("Display")] = "display"

wo_ogsl_1["vertical"][wo_ogsl_1['order_product_name'].str.contains("Sale")] = "sale event products"

wo_ogsl_1["vertical"][wo_ogsl_1['order_product_name'].str.contains("Sale Event")] = "sale event products"

wo_ogsl_1["vertical"][wo_ogsl_1['order_product_name'].str.contains("Investment")] = "managed funds"

wo_ogsl_1["vertical"][wo_ogsl_1['order_product_name'].str.contains("Call")] = "at call accounts"

/Users/indraja/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/indraja/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/indraja/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/indraja/anaconda

In [14]:
company_uuid_no_osli=pd.read_csv('./datasets/company_uuid_match.csv')
company_uuid_no_osli=company_uuid_no_osli[['order_product_number','product_uuid']]

In [15]:
wo_ogsl=pd.merge(wo_ogsl_1,company_uuid_no_osli, on='order_product_number',how='left')

In [16]:
data_merged=pd.concat([df1, wo_ogsl], axis=0,ignore_index= True).drop_duplicates().reset_index(drop=True)

In [17]:
import numpy as np
data_merged['order_signed_date']=data_merged['order_signed_date'].replace([np.NaN,np.nan,'NaN','nan','Null','null'],'')
data_merged['active'] = np.where(data_merged['order_signed_date'].isnull(), 'False', 'True')

In [18]:
data_merged['capped'] = np.where(data_merged['capped']==1, True, False)

In [19]:
import string
data_merged['monetisation_type']= data_merged['order_product_name'].str.split().str[-1]
# data_merged['monetisation_type']=[x.strip().replace('(' ,'') for x in data_merged['monetisation_type']]
# data_merged['monetisation_type']=[x.strip().replace(')' ,'') for x in data_merged['monetisation_type']]
data_merged['monetisation_type']=data_merged['monetisation_type'].replace('(SOV)','SOV')


In [20]:
data_merged['CPC_price']=np.where(data_merged['monetisation_type']=='CPC',data_merged['p_unit'],'')
data_merged['CPA_price']=np.where(((data_merged['monetisation_type']=='CPA')|(data_merged['monetisation_type']=='CPA/CPAA')),data_merged['p_cpa'],'')
data_merged['CPL_price']=np.where(data_merged['monetisation_type']=='CPL',data_merged['p_cpl'],'')
data_merged['CPAA_price']=np.where(((data_merged['monetisation_type']=='CPAA')|(data_merged['monetisation_type']=='CPA/CPAA')),data_merged['p_cpaa'],'')

In [21]:
data_merged['sov_percentage']=data_merged['order_product_name'].str.extract('(\d+)', expand=False)

In [22]:
data_merged['osli_number']=data_merged['osli_number'].astype(str)
data_merged['osli_number']=[x.strip().replace(' ', '') for x in data_merged['osli_number']]

In [23]:
company_uuid=pd.read_csv('./datasets/company_uuid_match.csv')
# company_uuid=company_uuid[['company_uuid','product_uuid','osli_id']]
company_uuid=company_uuid[['company_uuid','product_uuid']]

In [24]:
data_merged=pd.DataFrame(data_merged)

In [25]:
# data=pd.merge(data_merged,company_uuid, on=['product_uuid','osli_id'],how='left')
data=pd.merge(data_merged,company_uuid, on=['product_uuid'],how='left')

In [26]:
# data['company_uuid_x']=data['company_uuid_x'].astype(str)
# data['company_uuid_y']=data['company_uuid_y'].astype(str)
# data['company_uuid_x'] = data['company_uuid_x'].replace('nan', '')
# data['company_uuid_y'] = data['company_uuid_y'].replace('nan', '')
# data['company_uuid']=data[['company_uuid_x','company_uuid_y']].astype(str).sum(axis=1)
data['company_uuid']=data['company_uuid'].astype(str)
data['company_uuid'] = data['company_uuid'].replace(['nan','NaN',np.nan,np.NaN], '')

In [27]:
import numpy as np
data['osli_number'] = data['osli_number'].replace(['nan', 'None'], '')

In [28]:
data1=data.replace(np.nan,'',regex=True)

In [29]:

data1['vertical'] = data1['vertical'].replace('Home Loans', 'home loans')
data1['vertical'] = data1['vertical'].replace('Car Loans', 'car loans')
data1['vertical'] = data1['vertical'].replace('Personal Loans', 'personal loans')
data1['vertical'] = data1['vertical'].replace('Term Deposits', 'term deposits')
data1['vertical'] = data1['vertical'].replace('Credit Cards', 'credit cards')
data1['vertical'] = data1['vertical'].replace('Car Insurance', 'car insurance')
data1['vertical'] = data1['vertical'].replace('Managed Funds', 'managed funds')
data1['vertical'] = data1['vertical'].replace('Savings A/C', 'savings accounts')
data1['vertical'] = data1['vertical'].replace('Transaction A/C', 'transaction accounts')
data['vertical'] = data['vertical'].replace('Transaction accounts', 'transaction accounts')
data1['vertical'] = data1['vertical'].replace('Pension Funds', 'pension')
data1['vertical'] = data1['vertical'].replace('Pension', 'pension')
data1['vertical'] = data1['vertical'].replace('Superannuation', 'superannuation')
data1['vertical'] = data1['vertical'].replace('Managed Funds', 'managed funds')
data1['vertical'] = data1['vertical'].replace('Online Share Trading', 'online share trading')
data1['vertical'] = data1['vertical'].replace('Margin Loans', 'margin loans')
data1['vertical'] = data1['vertical'].replace('Foreign Exchange', 'foreign exchange')
data1['vertical'] = data1['vertical'].replace('Mortgage Insurance', 'mortgage insurance')
data1['vertical'] = data1['vertical'].replace('Home Insurance', 'home insurance')
data1['vertical'] = data1['vertical'].replace('Bad Credit', 'bad credit')
data1['vertical'] = data1['vertical'].replace('Business Loans', 'business loans')
data1['vertical'] = data1['vertical'].replace('Health Product', 'health product')
data1['vertical'] = data1['vertical'].replace('Income Protection Insurance', 'income protection insurance')
data1['vertical'] = data1['vertical'].replace('Life Insurance', 'life insurance')
data1['vertical'] = data1['vertical'].replace('Display ', 'display')
data1['vertical'] = data1['vertical'].replace('Sale Event Products ', 'sale event products')
data1['vertical'] = data1['vertical'].replace('Investment Funds ', 'managed funds')

In [30]:
from time import gmtime, strftime
data1['datetime']= strftime("%Y-%m-%d %H:%M:%S", gmtime())
data1['updated_at']=strftime("%Y-%m-%d %H:%M:%S", gmtime())

In [31]:
data1=data1[['osli_id','date_start','date_end','vertical','order_number','order_signed_date','active',
                         'order_effective_date','order_end_date','authorized_by_name','owner_name','osli_number',
                         'console_product_name','order_product_number','order_product_name','company_uuid',
                         'company_name','product_uuid','monetisation_type','sov_percentage','CPC_price','CPA_price',
                         'CPAA_price','CPL_price','number_of_cpa','number_of_cpaa','number_of_cpl',
                         'actual_clicks','billed_clicks','excess_clicks','capped','budget','budget_consumption',
                         'gross_revenue','agency_name','agency_percentage','agency_commission','product_url',
                          'p_unit','p_cpa','p_cpaa','p_cpl','updated_at','datetime'
                       ]]

In [32]:
from datetime import date
#type casting all the columns(since it should match with the column types in DW)
data1['osli_id']=data1['osli_id'].astype(str)
data1['date_start']=data1['date_start'].astype(date)
data1['date_end']=data1['date_end'].astype(date)
data1['vertical']=data1['vertical'].astype(str)
# data1['order_number']=data1['order_number'].astype(long)
data1['order_signed_date']=data1['order_signed_date'].astype(date)
data1['active']=data1['active'].astype(bool)
data1['order_effective_date']=data1['order_effective_date'].astype(date)
data1['order_end_date']=data1['order_end_date'].astype(date)
data1['authorized_by_name']=data1['authorized_by_name'].astype(str)
data1['owner_name']=data1['owner_name'].astype(str)
data1['osli_number']=data1['osli_number'].astype(str)
data1['console_product_name']=data1['console_product_name'].astype(str)
data1['order_product_name']=data1['order_product_name'].astype(str)
data1['company_uuid']=data1['company_uuid'].astype(str)
data1['company_name']=data1['company_name'].astype(str)
data1['product_uuid']=data1['product_uuid'].astype(str)
data1['monetisation_type']=data1['monetisation_type'].astype(str)
data1['sov_percentage']=data1['sov_percentage'].astype(str)
data1['CPC_price']=data1['CPC_price'].astype(str)
data1['CPA_price']=data1['CPA_price'].astype(str)
data1['CPAA_price']=data1['CPAA_price'].astype(str)
data1['CPL_price']=data1['CPL_price'].astype(str)
data1['number_of_cpa']=data1['number_of_cpa'].astype(str)
data1['number_of_cpaa']=data1['number_of_cpaa'].astype(str)
data1['number_of_cpl']=data1['number_of_cpl'].astype(str)
data1['actual_clicks']=data1['actual_clicks'].astype(str)
data1['billed_clicks']=data1['billed_clicks'].astype(str)
data1['excess_clicks']=data1['excess_clicks'].astype(str)
data1['capped']=data1['capped'].astype(bool)
data1['budget']=data1['budget'].astype(str)
data1['budget_consumption']=data1['budget_consumption'].astype(str)
data1['gross_revenue']=data1['gross_revenue'].astype(str)
data1['agency_name']=data1['agency_name'].astype(str)
data1['agency_percentage']=data1['agency_percentage'].astype(str)
data1['agency_commission']=data1['agency_commission'].astype(str)
data1['product_url']=data1['product_url'].astype(str)
data1['p_unit']=data1['p_unit'].astype(str)
data1['p_cpa']=data1['p_cpa'].astype(str)
data1['p_cpaa']=data1['p_cpaa'].astype(str)
data1['p_cpl']=data1['p_cpl'].astype(str)
data1['updated_at']=data1['updated_at'].astype(date)
data1['datetime']=data1['datetime'].astype(date)

In [33]:
data1['vertical'] = data1['vertical'].replace('Transaction accounts', 'transaction accounts')

In [34]:
# data1.to_csv('orders_table_Aug000.csv', sep=',', encoding='utf-8',index=None)

In [35]:
data2=pd.read_csv('orders_table_Aug0000.csv')

In [36]:
my_data=pd.concat([data1,data2]).drop_duplicates(subset=['order_number','order_product_number'],keep='last').reset_index(drop=True)
# df.drop_duplicates(df.columns.difference(['Description']))

In [37]:
my_data=my_data[['osli_id','date_start','date_end','vertical','order_number','order_signed_date','active',
                         'order_effective_date','order_end_date','authorized_by_name','owner_name','osli_number',
                         'console_product_name','order_product_number','order_product_name','company_uuid',
                         'company_name','product_uuid','monetisation_type','sov_percentage','CPC_price','CPA_price',
                         'CPAA_price','CPL_price','number_of_cpa','number_of_cpaa','number_of_cpl',
                         'actual_clicks','billed_clicks','excess_clicks','capped','budget','budget_consumption',
                         'gross_revenue','agency_name','agency_percentage','agency_commission','product_url',
                          'p_unit','p_cpa','p_cpaa','p_cpl','updated_at','datetime'
                       ]]

In [38]:
my_data['order_number'].isnull().value_counts()

False    472
Name: order_number, dtype: int64

In [39]:
my_data.to_csv('orders_table_Sep0000.csv', sep=',', encoding='utf-8',index=None)